# Drip Drip

### Imports, Settings, Constants

In [1]:
from datetime import date, timedelta
import pandas as pd
import warnings
from  yahoo_fin import stock_info

# helps with math for incrementals
PERIODS = {
    'quarterly': {'days': 91, 'num': 4},
    'biannually': {'days': 182, 'num': 2},
    'annually': {'days': 365, 'num': 1}
}

# pandas formatting
summary_formats = {
    'initial': '${:,.2f}',
    'incremental': '${:,.2f}',
    'total': '${:,.2f}',
    'div_payout': '${:,.2f}',
    'div_value': '${:,.2f}',
    'personally_invested': '${:,.2f}',
    'cost_basis': '${:,.2f}',
    'days_invested': '{:,}'
}

table_formats = {
    'open_price': '${:,.2f}', 
    'open_shares': '{:,.2f}',  
    'new_inc_shares': '{:,.2f}', 
    'new_div_shares': '{:,.2f}',
    'close_shares': '{:,.2f}',
    'div_money_per_share': '${:,.2f}', 
    'div_payout': '${:,.2f}', 
    'inc_invest': '${:,.2f}'
}

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

### Functions

In [2]:
def get_ticker_data(ticker: str) -> pd.DataFrame:
    """
    The output of this is almost the same as the "_data" excel sheets but has quote data for every
    day and not just the ones with dividend payouts.
    """
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        quotes = stock_info.get_data(ticker).drop(columns=["ticker"])
        dividends = stock_info.get_dividends(ticker).drop(columns=["ticker"])
    
    full_df = quotes.join(dividends, how='left')
    full_df.index.name = 'date'
    return full_df
    

def get_tickers(url: str = "excel/aristocrats.csv") -> pd.DataFrame:
    """
    Just reads in a CSV list of securities. Defaults to that one you sent me.
    """
    tickers = pd.read_csv(url)
    tickers['ticker'] = tickers['ticker'].str.lower()
    return tickers.set_index("ticker")


def get_percent(num):
    """formats percents as strings with the %"""
    return f"{str(round(num * 100, 2))}%"


def find_next_market_day(d: date, quote_days: pd.DataFrame) -> date:
    """
    Fasts forward and returns the next market-open day for when the code requests a weekend or holiday.
    """
    return quote_days[d:d+timedelta(days=3)].iloc[0].name.date()


def find_incremental_day(d: date, quote_days: pd.DataFrame) -> date:
    """
    This returns a dividend day if there is one within +/- 25 days. 
    
    I wanted to overlap them because it made it easier to test against the workbook. 
    """
    div_window = quote_days[d-timedelta(days=25):d+timedelta(days=25)]
    div_day = div_window.loc[div_window['dividend'] > 0]
    if len(div_day) > 0:
        return div_day.iloc[0].name.date()
    return find_next_market_day(d, quote_days)


def get_incremental_days(start: date, end: date, quotes_df: pd.DataFrame, frequency: str) -> [date]:
    """
    These are kinda ugly but it's just building a list of dates for incremental buys that are either 365, 182 or 91 days apart.
    """
    num_days = int((end-start).days/PERIODS[frequency]['days']) + 1
    days = [start + timedelta(days=(x * PERIODS[frequency]['days'])) for x in range(num_days)]
    return [find_incremental_day(x, quotes_df) for x in days]


def get_analysis(ticker: str, buy_date: date, inc_start_date: date, end_date: date, initial_amt: int, inc_amt: int, freq: str = 'quarterly'):
    """
    The output of this one looks like the bottom table on the "Stock#" sheets in excel.

    basically adds a column to the quotes/dividend dataframe for incremental buys then filters to days with only divs or incrementals
    then loops through that while keeping a running counter of how many shares are owned.

    it makes sense when you print out the table.
    """
    
    quotes_df = get_ticker_data(ticker)

    market_buy_date = find_next_market_day(buy_date, quotes_df)
    market_sell_date = find_next_market_day(end_date, quotes_df)
    
    quotes_df = quotes_df.loc[market_buy_date:market_sell_date]
    quotes_df.loc[get_incremental_days(inc_start_date, market_sell_date, quotes_df, freq),'incremental'] = inc_amt / PERIODS[freq]['num']

    shares = initial_amt / quotes_df.iloc[0]['open']

    out_list = [{
        'date': quotes_df.iloc[0].name.date(),
        'open_price': quotes_df.iloc[0]['open'],
        'open_shares': 0,
        'new_inc_shares': 0,
        'new_div_shares': 0,
        'close_shares': shares,
    }]

    events_df = quotes_df.dropna(subset=['incremental', 'dividend'], how='all').fillna(0)

    for d, ev in events_df.iterrows():
        div_payout = shares * ev['dividend']
        new_div_shares = div_payout / ev['open']
        new_inc_shares = ev['incremental'] / ev['open']
        out_list.append({
            'date': d.date(), 
            'open_price': ev['open'], 
            'open_shares': shares,  
            'new_inc_shares': new_inc_shares, 
            'new_div_shares': new_div_shares,
            'close_shares': shares + new_div_shares + new_inc_shares,
            'div_money_per_share': ev['dividend'], 
            'div_payout': div_payout, 
            'inc_invest': ev['incremental']
        })
        shares += new_div_shares + new_inc_shares

    out_list.append({
        'date': quotes_df.iloc[-1].name.date(), 
        'open_price': quotes_df.iloc[-1]['open'],
        'open_shares': shares,
        'close_shares': 0, 
    })

    return pd.DataFrame(out_list).fillna(0).set_index('date')


def get_summary(df, ticker):
    """
    this takes the table at the bottom of the "stock#" sheets and creates the summary table at the top.
    just does all the aggregation.
    """
    buy = df.iloc[0]
    sell = df.iloc[-1]

    initial_investment = buy['close_shares'] * buy['open_price']
    personally_invested = df['inc_invest'].sum() + initial_investment
    div_value = df['new_div_shares'].sum() * sell['open_price']
    div_payout = df['div_payout'].sum()
    inc_inv = df['inc_invest'].max()
    total_value = sell['open_shares'] * sell['open_price']
    cost_basis = personally_invested + div_payout

    return pd.Series({
        'security': ticker,
        'initial': initial_investment,
        'incremental': inc_inv,
        'div_value': div_value,
        'dividend_return': get_percent((div_value - div_payout) / div_payout),
        'div_payout': div_payout,
        'personally_invested': personally_invested,
        'cost_basis': cost_basis,
        'total': total_value,
        'return': get_percent((total_value - cost_basis) / cost_basis),
        'personal_return': get_percent((total_value - personally_invested) / personally_invested),
        'days_invested': (sell.name - buy.name).days  
    })


def run_list(stock_list):
    """
    just a wrapper. takes a list of stocks then runs the calc on all of them and adds all their summaries to a data frame
    """
    out_list = []
    for l in stock_list:
        n = l.pop('name')
        df = get_analysis(**l)
        out_list.append(get_summary(df, f"{n} ({l['ticker'].upper()})"))

    return pd.DataFrame(out_list)

## Run it

### Validating vs. excel

In [3]:
# matches - rounding errors

print("#1 Lockheed: ")
df = get_analysis('lmt', date(1995,5,25), date(1995,5,25), date(2021,11,30), 2500, 6000)
display(get_summary(df, 'lmt'))
display(df.style.format(table_formats))

#1 Lockheed: 


security                       lmt
initial                   2,500.00
incremental               1,500.00
div_value               511,335.30
dividend_return             81.81%
div_payout              281,239.34
personally_invested     163,000.00
cost_basis              444,239.34
total                 1,314,976.22
return                     196.01%
personal_return            706.73%
days_invested                 9686
dtype: object

,open_price,open_shares,new_inc_shares,new_div_shares,close_shares,div_money_per_share,div_payout,inc_invest
date,,,,,,,,
1995-05-25,$29.75,0.00,0.00,0.00,84.03,$0.00,$0.00,$0.00
1995-05-25,$29.75,84.03,50.42,0.49,134.95,$0.17,$14.71,"$1,500.00"
1995-08-30,$30.06,134.95,49.90,0.79,185.63,$0.17,$23.62,"$1,500.00"
1995-11-29,$36.62,185.63,40.96,0.89,227.47,$0.17,$32.49,"$1,500.00"
1996-02-29,$38.25,227.47,39.22,1.19,267.88,$0.20,$45.49,"$1,500.00"
1996-05-30,$41.12,267.88,36.47,1.30,305.65,$0.20,$53.58,"$1,500.00"
1996-08-29,$43.06,305.65,34.83,1.42,341.91,$0.20,$61.13,"$1,500.00"
1996-11-27,$45.62,341.91,32.88,1.50,376.28,$0.20,$68.38,"$1,500.00"
1997-03-06,$44.00,376.28,34.09,1.71,412.08,$0.20,$75.26,"$1,500.00"


In [ ]:
# i think the sheet has the wrong quote info for coca cola...

print("#2 Coca Cola: ")
df = get_analysis('ko', date(2016, 4, 12), date(2016, 4, 12), date(2021, 10, 8), 15000, 0)
display(get_summary(df,'ko'))
display(df.style.format(table_formats))

In [ ]:
# matches - rounding errors

print("#3 Exxon: ")
df = get_analysis('xom', date(2013,2,7), date(2013,2,7), date(2023,1,31), 15000, 0)
display(get_summary(df,'xom'))
display(df.style.format(table_formats))

In [ ]:
# matches - rounding errors

print("#4 Chevron: ")
df = get_analysis('cvx', date(2013,2,13), date(2013,2,13), date(2023,1,26), 15000, 0)
display(get_summary(df, 'cvx'))
display(df.style.format(table_formats))

In [ ]:
# matches - rounding errors

print("#5 IBM: ")
ibm_df = get_analysis('ibm', date(2013,2,6), date(2013,2,6), date(2023,1,26), 15000, 6000)
display(get_summary(ibm_df, 'ibm'))
display(ibm_df.style.format(table_formats))

In [ ]:
# matches - rounding errors

print("#6 Altria: ")
df = get_analysis('mo', date(2013, 3, 13), date(2013, 3, 13), date(2023,6,14) 15000, 0)
display(get_summary(df, 'mo'))
display(df.style.format(table_formats))

In [ ]:
# I added this for medtronic just cause i saw it in the excel sheet without data lol

print("#7 Medtronic: ")
df = get_analysis('mdt', date(2013,1,25), date(2018,2,20), date(2023,8,11), 15000, 6000)
display(get_summary(df, 'mdt'))
display(df.style.format(table_formats))

### Summary page

This section outputs basically the top part of the Summary excel sheet to mess around with percentages.

In [4]:
initial_investment = 15000
number_stocks = 6

yearly_roth_limit = 6000
freq = 'quarterly'

percentages = {
    'lmt':       .25
    ,'ko':       0
    ,'xom':      .1
    ,'cvx':      .1
    ,'ibm':      .3
    ,'mo':       .5
}

test_stocks = [
    {'ticker': 'lmt', 'name': 'Lockheed Martin', 'buy_date': date(1995,5,25), 'inc_start_date': date(1995,5,25), 'end_date': date(2021,11,30), 'initial_amt': initial_investment/number_stocks, 'inc_amt': yearly_roth_limit * percentages['lmt'], 'freq': freq},
    {'ticker': 'ko', 'name': 'Coca-Cola', 'buy_date': date(2016,4,12), 'inc_start_date': date(2016,4,12), 'end_date': date(2021,10,8), 'initial_amt': initial_investment/number_stocks, 'inc_amt': yearly_roth_limit * percentages['ko'], 'freq': freq},
    {'ticker': 'xom', 'name': 'Exxon', 'buy_date': date(2013,2,7), 'inc_start_date': date(2013,2,7), 'end_date': date(2023,1,31), 'initial_amt': initial_investment/number_stocks, 'inc_amt': yearly_roth_limit * percentages['xom'], 'freq': freq},
    {'ticker': 'cvx', 'name': 'Chevron', 'buy_date': date(2013,2,13), 'inc_start_date': date(2013,2,13), 'end_date': date(2023,1,26), 'initial_amt': initial_investment/number_stocks, 'inc_amt': yearly_roth_limit * percentages['cvx'], 'freq': freq},
    {'ticker': 'ibm', 'name': 'IBM', 'buy_date': date(2013,2,6), 'inc_start_date': date(2013,2,6), 'end_date': date(2023,1,26), 'initial_amt': initial_investment/number_stocks, 'inc_amt': yearly_roth_limit * percentages['ibm'], 'freq': freq},
    {'ticker': 'mo', 'name': 'Altria', 'buy_date': date(2013, 3, 13), 'inc_start_date': date(2013, 3, 13), 'end_date': date(2023,6,14), 'initial_amt': initial_investment/number_stocks, 'inc_amt': yearly_roth_limit * percentages['mo'], 'freq': freq}
]

summary_list = []
for t in test_stocks:
    disp_name = f"{t['name']} ({t['ticker'].upper()})"
    df = get_analysis(t['ticker'], t['buy_date'], t['inc_start_date'], t['end_date'], t['initial_amt'], t['inc_amt'], t['freq'])
    summary_list.append(get_summary(df, disp_name))

display(pd.DataFrame(summary_list).style.format(summary_formats))

,security,initial,incremental,div_value,dividend_return,div_payout,personally_invested,cost_basis,total,return,personal_return,days_invested
0,Lockheed Martin (LMT),"$2,500.00",$375.00,"$147,217.81",84.84%,"$79,647.39","$42,625.00","$122,272.39","$369,380.14",202.1%,766.58%,"9,686"
1,Coca-Cola (KO),"$2,500.00",$0.00,$568.82,12.3%,$506.52,"$2,500.00","$3,006.52","$3,477.68",15.67%,39.11%,"2,005"
2,Exxon (XOM),"$2,500.00",$150.00,"$3,929.12",60.12%,"$2,453.83","$8,650.00","$11,103.83","$16,212.46",46.01%,87.43%,"3,645"
3,Chevron (CVX),"$2,500.00",$150.00,"$4,511.31",62.9%,"$2,769.42","$8,500.00","$11,269.42","$18,491.54",64.09%,117.55%,"3,634"
4,IBM (IBM),"$2,500.00",$450.00,"$5,038.99",3.29%,"$4,878.70","$20,950.00","$25,828.70","$24,755.32",-4.16%,18.16%,"3,641"
5,Altria (MO),"$2,500.00",$750.00,"$15,198.95",-6.29%,"$16,219.97","$34,000.00","$50,219.97","$47,729.59",-4.96%,40.38%,"3,745"


### Run em all

In [ ]:
ticks = get_tickers()
summary_list = []

initial_buy = date(2013,1,5)
incremental_start = date(2018,2,20)
sell_date = date(2023,8,11)

for t, info in ticks.iterrows():
    disp_name = f"{info['name']} ({t.upper()})"
    df = get_analysis(t, initial_buy, incremental_start, sell_date, 15000, 6000)
    summary_list.append(get_summary(df, disp_name))

summary_df = pd.DataFrame(summary_list)

In [ ]:
summary_df.sort_values(by=['dividend_return']).tail(25).style.format(summary_formats)